In [9]:
import pandas as pd
from decimal import Decimal

from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-5

In this demo we test that y != y_int is supported upon strategy creation.

In [10]:
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

In [11]:
test_orders = pd.read_csv('256 orders with curr rate alternating.csv', dtype=float)
test_orders.rename(columns={'lowest-rate':'lowest_rate', 'highest-rate':'highest_rate', 'current-rate':'current_rate'}, inplace=True)
test_orders['one_on_lowest_rate'] = 1/test_orders.lowest_rate
test_orders['one_on_highest_rate'] = 1/test_orders.highest_rate
test_orders['one_on_current_rate'] = 1/test_orders.current_rate
test_orders['y_int'] = [test_orders.liquidity[i] * (test_orders.one_on_highest_rate[i] - test_orders.one_on_lowest_rate[i]) / (test_orders.one_on_current_rate[i] - test_orders.one_on_lowest_rate[i]) for i in test_orders.index]
test_orders['fix_trade'] = test_orders.y_int - test_orders.liquidity
test_orders['y_int_minus_fixtrade'] = test_orders.y_int - test_orders.fix_trade
test_orders

,id,liquidity,lowest_rate,highest_rate,current_rate,one_on_lowest_rate,one_on_highest_rate,one_on_current_rate,y_int,fix_trade,y_int_minus_fixtrade
0,0.0,254814732.0,1255.0,2255.0,1256.0,0.000797,0.000443,0.000796,1.419279e+11,1.416730e+11,254814732.0
1,1.0,253827078.0,1254.0,2254.0,2254.0,0.000797,0.000444,0.000444,2.538271e+08,0.000000e+00,253827078.0
2,2.0,252839424.0,1253.0,2253.0,1753.0,0.000798,0.000444,0.000570,3.934554e+08,1.406160e+08,252839424.0
3,3.0,251851770.0,1252.0,2252.0,1253.0,0.000799,0.000444,0.000798,1.401289e+11,1.398770e+11,251851770.0
4,4.0,6913578.0,1004.0,2004.0,1504.0,0.000996,0.000499,0.000665,1.037727e+07,3.463689e+06,6913578.0
...,...,...,...,...,...,...,...,...,...,...,...
251,251.0,250864116.0,1251.0,2251.0,2251.0,0.000799,0.000444,0.000444,2.508641e+08,2.980232e-08,250864116.0
252,252.0,5925924.0,1003.0,2003.0,1004.0,0.000997,0.000499,0.000996,2.970358e+09,2.964432e+09,5925924.0
253,253.0,4938270.0,1002.0,2002.0,2002.0,0.000998,0.000500,0.000500,4.938270e+06,0.000000e+00,4938270.0
254,254.0,3950616.0,1001.0,2001.0,1501.0,0.000999,0.000500,0.000666,5.926911e+06,1.976295e+06,3950616.0


In [12]:
for i in test_orders.index:
    FastSim.add_strategy(
        tkn='ETH',
        amt_sell=Decimal(test_orders.y_int[i]),
        psell_start=1/Decimal(test_orders.highest_rate[i]),
        psell_end=1/Decimal(test_orders.lowest_rate[i]),
        pbuy_start=None,
        pbuy_end=None,
        pair='ETH/USDC',
        psell_marginal=1/Decimal(test_orders.current_rate[i]),
        pbuy_marginal=None
    )

FastSim.orders
results = FastSim.state()['orders']
results['y_eq'] = results['y_int']==results['y']
results = results[results['y_eq']==False]
results

{0: Order(B=35.4259791678367, S=12.060861114498366, y_int=Decimal('141927850728162.017822265625'), p_high=Decimal('2255.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), p_low=Decimal('1255.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), p_marginal=Decimal('1256.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), disabled=False, _y=Decimal('141927850728.162017822265625'), D=None, _C=None, pair=P('ETH/USDC'), tkn='ETH', pair_name='ETHUSDC', adj_n_mode=False, auto_convert_variables=True, id=0, linked_to_id=1, _reverseq=False, verbose=False),
 1: Order(B=0.0, S=0.0, y_int=Decimal('0'), p_high=Decimal('0'), p_low=Decimal('0'), p_marginal=None, disabled=True, _y=Decimal('0'), D=Decimal('0'), _C=Decimal('0'), pair=P('ETH/USDC'), tkn='USDC', pair_name='ETHUSDC', 